In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2525444/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_20_9_10,0.998164,0.984939,0.996820,0.992676,0.540562,2.110117,0.899920,1.540612,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
1,model_20_9_11,0.998163,0.984585,0.996670,0.992456,0.540760,2.159714,0.942428,1.586876,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
2,model_20_9_12,0.998157,0.984248,0.996530,0.992249,0.542761,2.206867,0.981907,1.630417,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
3,model_20_9_9,0.998156,0.985307,0.996981,0.992908,0.542856,2.058500,0.854375,1.491852,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
4,model_20_9_13,0.998145,0.983931,0.996401,0.992055,0.546046,2.251374,1.018429,1.671165,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
...,...,...,...,...,...,...,...,...,...,...
1993,model_11_4_1,0.932746,0.936307,0.963064,0.957236,19.801823,9.391358,11.695643,10.475728,"Hidden Size=[20, 7], regularizer=0.01, learnin..."
2005,model_13_6_1,0.932156,0.980891,0.932961,0.972775,19.975702,4.101117,9.981978,6.868582,"Hidden Size=[20, 10], regularizer=0.01, learni..."
2008,model_15_9_3,0.931832,0.977859,0.978555,0.978064,20.071014,11.931387,5.430683,8.872230,"Hidden Size=[30, 5], regularizer=0.01, learnin..."
2020,model_6_6_1,0.931332,0.989507,0.987396,0.988460,20.218147,2.199469,4.764439,3.406517,"Hidden Size=[15, 7], regularizer=0.1, learning..."
